<a href="https://colab.research.google.com/github/Zia-Ul-Hasan/NLP_SEMACNTIC_PRODUCT_SEARCH/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install streamlit sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 121.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [6]:
import os
os.makedirs("my_model", exist_ok=True)
import shutil

files = [
    "model.safetensors",
    "config.json",
    "tokenizer_config.json",
    "vocab.txt",
    "config_sentence_transformers.json"
]

for f in files:
    shutil.move(f, os.path.join("my_model", f))



In [11]:
pip install streamlit-jupyter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.15
    Uninstalling jupyterlab_widgets-3.0.15:
      Successfully uninstalled jupyterlab_widgets-3.0.15
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-cli

In [10]:
import streamlit as st
import numpy as np
import pandas as pd
import re
from sentence_transformers import SentenceTransformer

# Load model
@st.cache_resource
def load_model():
    model = SentenceTransformer('my_model')  # Load from directory
    return model

# Load data
@st.cache_data
def load_data():
    product_df = pd.read_csv('/content/product.csv', sep='\t')
    product_embeddings = np.load('product_embeddings_full.npy')
    return product_df, product_embeddings

model = load_model()
product_df, product_embeddings = load_data()

# Text cleaning
def clean_text(text):
    return re.sub(r'[^a-z0-9\s]', ' ', str(text).lower()).strip()

# Search logic
def get_top_matches(query, top_k=5):
    query = clean_text(query)
    query_vector = model.encode([query], normalize_embeddings=True)
    similarities = np.dot(product_embeddings, query_vector.T).squeeze()
    top_indices = similarities.argsort()[-top_k:][::-1]
    return product_df.iloc[top_indices]

# Streamlit UI
st.title("🔎 Semantic Product Search")

query = st.text_input("Enter your product search query:")

if query:
    results = get_top_matches(query)
    st.subheader("Top Matches:")
    st.dataframe(results)


2025-06-02 05:16:46.848 No runtime found, using MemoryCacheStorageManager
2025-06-02 05:16:46.850 No runtime found, using MemoryCacheStorageManager
2025-06-02 05:16:46.852 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 05:16:46.853 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 05:16:46.854 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 05:16:47.357 Thread 'Thread-15': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 05:16:47.358 Thread 'Thread-15': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 05:16:51.820 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 05:16:51.822 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored